In [35]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn import model_selection 
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
df = pd.read_csv("Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[["League","Division","NewLeague"]])
y = df["Salary"]
X_ = df.drop(["Salary","League","Division","NewLeague"],axis=1).astype("float64")
X = pd.concat([X_,dms[["League_N","Division_W","NewLeague_N"]]],axis = 1)
X.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,1,0,1
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,0,1,0


In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state=42)

In [17]:
model = Lasso().fit(X_train,y_train)

C:\Users\ardaa_iyfsapc\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8256653.399850385, tolerance: 3898.686956380658
  positive)


In [19]:
model.intercept_

-5.587450677336733

In [21]:
model.coef_

array([-1.74875691e+00,  8.59204135e+00,  6.67993798e+00, -3.06715333e+00,
       -1.91843070e+00,  5.32372890e+00,  8.39184117e+00, -1.63172447e-01,
       -8.22311277e-02, -3.93602861e-01,  1.71118530e+00,  6.55730545e-01,
       -6.48379405e-01,  2.59815358e-01,  2.73041157e-01, -4.41440454e-01,
        8.54474011e+01, -9.59701213e+01, -2.13086605e+01])

In [27]:
lasso = Lasso()
coefs = []
alphas = np.random.randint(0,1000,100)
for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X_train,y_train)
    coefs.append(lasso.coef_)

C:\Users\ardaa_iyfsapc\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4986.064965523779, tolerance: 3898.686956380658
  positive)
C:\Users\ardaa_iyfsapc\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6307.625522036105, tolerance: 3898.686956380658
  positive)
C:\Users\ardaa_iyfsapc\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6826.887084558606, tolerance: 3898.686956380658
  positive)
C:\Users\ardaa_iyfsapc\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the num

# Tahmin

In [30]:
model.predict(X_train)[:5]

array([377.26270596, 786.51524513, 495.14140718, 117.19492966,
       429.04228506])

In [31]:
model.predict(X_test)[:5]

array([ 609.18826367,  696.96810702, 1009.06157391,  412.22773375,
        409.25851712])

In [32]:
y_pred = model.predict(X_test)

In [36]:
np.sqrt(mean_squared_error(y_test,y_pred))

671.6440643451509

In [38]:
r2_score(y_test,y_pred) #açıklanma yüzdesi, verilerde ufak bir hata var (benim hatam) :) 

-1.0838618273576222

# Model Tuning

In [47]:
from sklearn.linear_model import LassoCV
lasso_cv = LassoCV(cv=10,max_iter = 10000).fit(X_train,y_train)

In [48]:
lasso_cv.alpha_

563.4670501833854

In [50]:
lasso_tuned = Lasso().set_params(alpha=lasso_cv.alpha_).fit(X_train,y_train)

In [51]:
y_pred = lasso_tuned.predict(X_test)

In [52]:
np.sqrt(mean_squared_error(y_test,y_pred))

373.5957225069793

In [54]:
pd.Series(lasso_tuned.coef_,index=X_train.columns)

AtBat         -0.000000
Hits           2.533141
HmRun          0.000000
Runs           0.000000
RBI            0.000000
Walks          0.520003
Years          0.000000
CAtBat        -0.188935
CHits          0.360895
CHmRun        -0.000000
CRuns          0.835617
CRBI           0.427816
CWalks         0.000527
PutOuts        0.279218
Assists        0.081443
Errors        -0.000000
League_N       0.000000
Division_W    -0.000000
NewLeague_N    0.000000
dtype: float64